In [1]:
from xgboost import XGBClassifier

import sys
from pathlib import Path
import pandas as pd 
import numpy as np
sys.path.append(str(Path("../../..").resolve()))
from src.constants import model_data_dir
from src.utils import make_target_class, use_next_day_target

In [2]:
df_train = pd.read_csv(model_data_dir / "train_classification.csv")
df_test = pd.read_csv(model_data_dir / "test_classification.csv")
test_dates = pd.to_datetime(df_test["date"])
df_test = df_test.drop("date", axis=1)
df_train = make_target_class(df_train)
df_test = make_target_class(df_test)

In [5]:
use_next_day_target(df_train, "y")

,date,vix,v2x,move,gold_vol,jpm_global_fx_vol,jpm_g7_fx_vol,jpm_em_fx_vol,eurusd_1m_vol,usdjpy_1m_vol,...,usd_libor_swaption_vol_rolling_40,s&p_500_vrp_rolling_40,s&p_500_skew_rolling_40,eurusd_risk_reversal_rolling_40,usdjpy_risk_reversal_rolling_40,eurjpy_risk_reversal_rolling_40,month,year,day,weekday
1,2000-07-27,-0.0305,-0.2347,0.0167,-0.5660,0.2132,0.4648,-0.7975,0.3823,0.2698,...,-0.6174,-1.0756,-0.9314,-0.2176,-0.1166,-0.0524,7,2000,27,3
2,2000-07-28,0.1082,-0.1098,0.0003,-0.5960,0.2247,0.4727,-0.7681,0.4299,0.2859,...,-0.6167,-1.0756,-0.9371,-0.2136,-0.1048,-0.0538,7,2000,28,4
3,2000-07-31,0.0970,-0.1387,0.0321,-0.5960,0.2477,0.5003,-0.7828,0.4934,0.3020,...,-0.6160,-1.0756,-0.9334,-0.2101,-0.0945,-0.0510,7,2000,31,0
4,2000-08-01,0.0758,-0.1185,0.0414,0.3059,0.2362,0.4885,-0.7828,0.4299,0.2859,...,-0.6160,-1.0756,-0.9269,-0.2067,-0.0857,-0.0499,8,2000,1,1
5,2000-08-02,0.0143,-0.1358,0.3105,0.1218,0.3244,0.5872,-0.7711,0.7633,0.3181,...,-0.6139,-1.0756,-0.9275,-0.2034,-0.0780,-0.0494,8,2000,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3374,2013-12-24,-0.8267,4.2335,-0.5860,-0.0560,-0.5578,-0.5455,-0.3563,-0.9052,-0.2861,...,-0.3441,-0.5041,0.3971,-0.2911,-0.2323,-0.2626,12,2013,24,1
3375,2013-12-26,-0.8434,4.1364,-0.6184,-0.0739,-0.5003,-0.4745,-0.3504,-0.8901,-0.1405,...,-0.3359,-0.4979,0.4254,-0.2935,-0.2309,-0.2627,12,2013,26,3
3376,2013-12-27,-0.8289,-0.6944,-0.6483,-0.1271,-0.4504,-0.4311,-0.2887,-0.7480,-0.0415,...,-0.3298,-0.4907,0.4523,-0.2941,-0.2283,-0.2627,12,2013,27,4
3377,2013-12-30,-0.7059,-0.6184,-0.5444,-0.1128,-0.4427,-0.4192,-0.2916,-0.5353,-0.0962,...,-0.3184,-0.4901,0.4620,-0.2940,-0.2251,-0.2629,12,2013,30,0


In [13]:
((y_train, x_train), (y_test, x_test)) = ((df_train.pop("value_1d_fwd_rel_d"), df_train), (df_test.pop("value_1d_fwd_rel_d"), df_test))

In [14]:
xgb_classifier = XGBClassifier(early_stopping_rounds=2)

In [15]:
xgb_classifier.fit(x_train, y_train,
                eval_set=[(x_test, y_test)])
y_pred = xgb_classifier.predict(x_test)

[0]	validation_0-logloss:0.69768
[1]	validation_0-logloss:0.70487


In [18]:
np.mean(y_pred == y_test)

0.49893955461293743